In [1]:
import pandas as pd
import numpy as np
import sys
import statistics as st
from google.colab import files

In [2]:
files.upload()

Saving pid.dat to pid.dat
Saving pidtest.dat to pidtest.dat


{'pid.dat': b'4,83,86,19,29.3,0.317,34,2\r\n6,105,70,32,30.8,0.122,37,2\r\n1,89,24,19,27.8,0.559,21,2\r\n6,111,64,39,34.2,0.26,24,2\r\n4,146,85,27,28.9,0.189,27,2\r\n0,127,80,37,36.3,0.804,23,2\r\n0,135,94,46,40.6,0.284,26,2\r\n2,100,68,25,38.5,0.324,26,2\r\n0,119,66,27,38.8,0.259,22,2\r\n3,142,80,15,32.4,0.2,63,2\r\n0,113,80,16,31,0.874,21,2\r\n3,113,44,13,22.4,0.14,22,2\r\n2,108,52,26,32.5,0.318,22,2\r\n4,144,58,28,29.5,0.287,37,2\r\n5,95,72,33,37.7,0.37,27,2\r\n2,92,76,20,24.2,1.698,28,2\r\n0,97,64,36,36.8,0.6,25,2\r\n0,173,78,32,46.5,1.159,58,2\r\n1,84,64,23,36.9,0.471,28,2\r\n4,112,78,40,39.4,0.236,38,2\r\n2,117,90,19,25.2,0.313,21,2\r\n1,101,50,15,24.2,0.526,26,2\r\n9,154,78,30,30.9,0.164,45,2\r\n1,73,50,10,23,0.248,21,2\r\n2,99,60,17,36.6,0.453,21,2\r\n3,111,58,31,29.5,0.43,22,2\r\n6,103,72,32,37.7,0.324,55,2\r\n3,115,66,39,38.1,0.15,28,2\r\n12,88,74,40,35.3,0.378,48,2\r\n5,139,64,35,28.6,0.411,26,2\r\n0,105,64,41,41.5,0.173,22,2\r\n4,129,86,20,35.1,0.231,23,2\r\n0,102,86,17,29.

In [6]:
# 데이터 불러오기
tr_data_name = input("Enter the name of training data file [(ex) pid.dat] : ")

tst_data_name = input("Enter the name of test data file [(ex) pidtest.dat] : ")

coding_fm = int(input("Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'):"))
                     
if(coding_fm == 1) : separator_fm = " "
else : separator_fm = ","

res_pos = int(input("Enter the column position of the response variable : [from 1 to p] : "))
                      
header= input("Does the data have column header? (True/False) : ")
                      
if(header == 'True'):
    trdata = pd.read_csv(tr_data_name, sep=separator_fm)
    tstdata = pd.read_csv(tst_data_name, sep=separator_fm)
else:
    trdata = pd.read_csv(tr_data_name, sep=separator_fm, header=None)
    tstdata = pd.read_csv(tst_data_name, sep=separator_fm, header=None)
                      
out_name=input("Enter the output file name to export [(ex) result.txt] : ")

Enter the name of training data file [(ex) pid.dat] : pid.dat
Enter the name of test data file [(ex) pidtest.dat] : pidtest.dat
Select the data coding format(1 = 'a b c' or 2 = 'a,b,c'):2
Enter the column position of the response variable : [from 1 to p] : 8
Does the data have column header? (True/False) : False
Enter the output file name to export [(ex) result.txt] : result.txt


In [7]:
# 주어진 코드
def My_CART():
    X_tr = trdata.drop(res_pos-1, axis=1)
    Y_tr = trdata[res_pos-1]
    X_tst = tstdata.drop(res_pos-1, axis=1)
    Y_tst = tstdata[res_pos-1]
    df = pd.DataFrame(columns = ['i','j','i_t1'])

    for i in range(X_tr.shape[1]):
        target_X = X_tr.iloc[:,i]
        unisort_dummy = np.sort(target_X.unique()) #*********************************************************************************
        for j in range(1, len(unisort_dummy)):
            vector_L = Y_tr[target_X < unisort_dummy[j]]
            vector_R = Y_tr[target_X >= unisort_dummy[j]]
            p_L1 = sum(vector_L == Y_tr.unique()[0]) / len(vector_L)
            p_L2 = sum(vector_L == Y_tr.unique()[1]) / len(vector_L)
            p_R1 = sum(vector_R == Y_tr.unique()[0]) / len(vector_R)
            p_R2 = sum(vector_R == Y_tr.unique()[1]) / len(vector_R)
            i_L = 1 - p_L1**2 - p_L2**2 # the gini impurity
            i_R = 1 - p_R1**2 - p_R2**2
            w_L = len(vector_L) / len(Y_tr) # weight
            w_R = len(vector_R) / len(Y_tr)
            i_t1 = i_L * w_L + i_R * w_R # 현재G - i_tl = goodness 
            df = df.append({'i': i, 'j': j, 'i_t1': i_t1}, ignore_index = True)

    answer_key = df.loc[df.iloc[:,2] == min(df.iloc[:,2]),:] # goodness가 최대 - i_t1이 최소
    answer_key = np.array(answer_key)[0]
    target_X = X_tr.iloc[:,int(answer_key[0])]
    unisort_dummy = np.sort(target_X.unique())
    vector_L = Y_tr[target_X < unisort_dummy[int(answer_key[1])]]
    vector_R = Y_tr[target_X >= unisort_dummy[int(answer_key[1])]]

    Prediction = []
    for i in range(len(Y_tst)):
        if X_tst.iloc[i, int(answer_key[0])] < np.sort(X_tr.iloc[:,int(answer_key[0])].unique())[int(answer_key[1])]:
            Y_pred = st.mode(vector_L)
        else:
            Y_pred = st.mode(vector_R)
        Prediction.append(Y_pred)

    accuracy_tst = sum(Prediction == Y_tst) / len(Y_tst)

    df = pd.DataFrame()
    df['Actual'] = Y_tst
    df['Prediction'] = Prediction
    df['count'] = np.ones(len(Y_tst))
    confusion_matrix = pd.pivot_table(df, values='count', index=['Actual'],
                                columns=['Prediction'], aggfunc=np.sum, fill_value=0)
    
    #sys.stdout = open(out_name,'w')
    print("Tree Structure (#Class{0}, #Class{1})".format(
        Y_tr.unique()[0], Y_tr.unique()[1]))
    print("Node 1: X{0} < {1} {2}".format(
        int(answer_key[0]) + 1, np.sort(X_tr.iloc[:,int(answer_key[0])].unique())[int(answer_key[1])],
        (sum(Y_tr == Y_tr.unique()[0]),sum(Y_tr == Y_tr.unique()[1]))))
    print("Node 2: {0} {1}".format(st.mode(vector_L),(sum(vector_L == Y_tr.unique()[0]), sum(vector_L == Y_tr.unique()[1]))))
    print("Node 3: {0} {1}".format(st.mode(vector_R),(sum(vector_R == Y_tr.unique()[0]), sum(vector_R == Y_tr.unique()[1]))))
    print("\nConfusion Matrix (Test)\n---------------------------")
    print(confusion_matrix)
    print('\nModel Summary (Test)\n---------------------------')        
    print("Overall Accuracy = {}".format(np.round(accuracy_tst, 3)))

In [8]:
My_CART()

Tree Structure (#Class2, #Class1)
Node 1: X2 < 147 (180, 72)
Node 2: 2 (164, 34)
Node 3: 1 (16, 38)

Confusion Matrix (Test)
---------------------------
Prediction   1    2
Actual             
1           37   50
2           11  128

Model Summary (Test)
---------------------------
Overall Accuracy = 0.73


In [25]:
def My_CART2():
    X_tr = trdata.drop(res_pos-1, axis=1)
    Y_tr = trdata[res_pos-1]
    X_tst = tstdata.drop(res_pos-1, axis=1)
    Y_tst = tstdata[res_pos-1]
    df = pd.DataFrame(columns = ['i','j','i_t1'])

    for i in range(X_tr.shape[1]):
        target_X = X_tr.iloc[:,i]
        unisort_dummy = np.sort(target_X.unique())
        unisort_dummy_2 = [(unisort_dummy[i] + unisort_dummy[i+1])/2 for i in range(0, len(unisort_dummy)-1)] #***********************************
        for j in range(1, len(unisort_dummy_2)):
            vector_L = Y_tr[target_X < unisort_dummy_2[j]] 
            vector_R = Y_tr[target_X >= unisort_dummy_2[j]]
            p_L1 = sum(vector_L == Y_tr.unique()[0]) / len(vector_L)
            p_L2 = sum(vector_L == Y_tr.unique()[1]) / len(vector_L)
            p_R1 = sum(vector_R == Y_tr.unique()[0]) / len(vector_R)
            p_R2 = sum(vector_R == Y_tr.unique()[1]) / len(vector_R)
            i_L = 1 - p_L1**2 - p_L2**2 
            i_R = 1 - p_R1**2 - p_R2**2
            w_L = len(vector_L) / len(Y_tr) 
            w_R = len(vector_R) / len(Y_tr)
            i_t1 = i_L * w_L + i_R * w_R 
            df = df.append({'i': i, 'j': j, 'i_t1': i_t1}, ignore_index = True)

    answer_key = df.loc[df.iloc[:,2] == min(df.iloc[:,2]),:] 
    answer_key = np.array(answer_key)[0]
    target_X = X_tr.iloc[:,int(answer_key[0])] 
    unisort_dummy = np.sort(target_X.unique())
    unisort_dummy_2 = [(unisort_dummy[i] + unisort_dummy[i+1])/2 for i in range(0, len(unisort_dummy)-1)]
    vector_L = Y_tr[target_X < unisort_dummy_2[int(answer_key[1])]]
    vector_R = Y_tr[target_X >= unisort_dummy_2[int(answer_key[1])]]

    Prediction = []
    for i in range(len(Y_tst)):
        if X_tst.iloc[i, int(answer_key[0])] < unisort_dummy_2[int(answer_key[1])]:
            Y_pred = st.mode(vector_L)
        else:
            Y_pred = st.mode(vector_R)
        Prediction.append(Y_pred)

    accuracy_tst = sum(Prediction == Y_tst) / len(Y_tst)

    df = pd.DataFrame()
    df['Actual'] = Y_tst
    df['Prediction'] = Prediction
    df['count'] = np.ones(len(Y_tst))
    confusion_matrix = pd.pivot_table(df, values='count', index=['Actual'],
                                columns=['Prediction'], aggfunc=np.sum, fill_value=0)
    
    #sys.stdout = open(out_name,'w')
    print("Tree Structure (#Class{0}, #Class{1})".format(
        Y_tr.unique()[0], Y_tr.unique()[1]))
    print("Node 1: X{0} < {1} {2}".format(
        int(answer_key[0]) + 1, unisort_dummy_2[int(answer_key[1])],
        (sum(Y_tr == Y_tr.unique()[0]),sum(Y_tr == Y_tr.unique()[1]))))
    print("Node 2: {0} {1}".format(st.mode(vector_L),(sum(vector_L == Y_tr.unique()[0]), sum(vector_L == Y_tr.unique()[1]))))
    print("Node 3: {0} {1}".format(st.mode(vector_R),(sum(vector_R == Y_tr.unique()[0]), sum(vector_R == Y_tr.unique()[1]))))
    print("\nConfusion Matrix (Test)\n---------------------------")
    print(confusion_matrix)
    print('\nModel Summary (Test)\n---------------------------')        
    print("Overall Accuracy = {}".format(np.round(accuracy_tst, 3)))

In [26]:
My_CART2()

Tree Structure (#Class2, #Class1)
Node 1: X2 < 146.5 (180, 72)
Node 2: 2 (164, 34)
Node 3: 1 (16, 38)

Confusion Matrix (Test)
---------------------------
Prediction   1    2
Actual             
1           37   50
2           11  128

Model Summary (Test)
---------------------------
Overall Accuracy = 0.73
